In [37]:
import regex as re
import os

from collections import defaultdict

import pandas as pd

DATA_DIR = '../ustawy'

# Load data

In [229]:
dataset = defaultdict(list)

for file_name in os.listdir(DATA_DIR):
    if re.match(r'.*\.txt$', file_name):
        with open(f'{DATA_DIR}/{file_name}', 'r', encoding='UTF-8') as file:
            bill = file.read()
            year_regex = r'u\s*s\s*t\s*a\s*w\s*a\s+' + r'\w*\s*\w*\s*\w*\s*\d*\s*' + r'z\s+dnia\s+\d+\s+\w+\s+' + r'(\d{4})'
            year_match = re.search(year_regex, bill, re.IGNORECASE)
            if year_match is None:
                print(f'Cannot read {file_name}')
            else:
                year = int(year_match.group(1))
                dataset[year].append(bill)
            
dataset_years = sorted(dataset.keys())
pd.DataFrame([(year, len(dataset[year])) for year in dataset_years], columns=['year', 'bill count'])

Cannot read 1996_400.txt


,year,bill count
0,1993,10
1,1994,41
2,1995,62
3,1996,56
4,1997,100
5,1998,80
6,1999,109
7,2000,161
8,2001,198
9,2002,7


# Utils

In [230]:
def count_by_year(p):
    matches = defaultdict(int)
    for year in dataset_years:
        for bill in dataset[year]:
            matches[year] += len(p.findall(bill))
            
    return matches

# Task 1

Based on https://lexlege.pl/zasady-techniki-prawodawczej/.

At first let's prepare a regex for a unit.

In [260]:
unit_regexes = [
    r'art\.',
    r'ust\.',
    r'pkt',
    r'lit\.'
]

unit_regex = r'(?:' + '|'.join(unit_regexes) + r')'
unit_regex

'(?:art\\.|ust\\.|pkt|lit\\.)'

## 1.1. Addition

Now we can create the addition regex. It seems that "dodaje się" is the only phrase to add a unit.

In [261]:
addition_pattern = re.compile(r'dodaje\s+się\s+' + unit_regex, re.IGNORECASE)
addition_pattern

regex.Regex('dodaje\\s+się\\s+(?:art\\.|ust\\.|pkt|lit\\.)', flags=regex.I | regex.V0)

In [263]:
addition_dict = count_by_year(addition_pattern)
# addition_dict

## 1.2. Removal